# Controlling the Output - Key Parameters

Hello and welcome back. We know how to send a prompt and get a response, but to truly engineer our outputs, we need to learn how to control the generation process itself. This is done by using optional parameters in our API calls.

These parameters are the control knobs for the AI. They let us fine-tune everything from creativity and length to format and stopping points. This notebook will provide a hands-on demonstration of the most important ones.

## Setup

Let's import the necessary libraries and define a few variables to avoid duplicating information.

In [ ]:
import json
import litellm
from dotenv import load_dotenv

load_dotenv()

MODEL_NAME = "openai/gpt-4o-mini"

def get_completion(messages, **kwargs):
    if "model" not in kwargs:
        kwargs["model"] = MODEL_NAME
        
    return litellm.completion(
        messages=messages,
        **kwargs
    )

## `temperature` - The Creativity Knob

`temperature` controls the randomness of the output. A low temperature (for example, 0.1) is deterministic and good for factual tasks. A high temperature (for example, 1.5) is creative and good for brainstorming.

Let's ask the model to brainstorm a name for a new Python testing library.

## `max_tokens` - The Safety Brake

`max_tokens` sets a hard limit on the number of tokens the model can generate. This is crucial for controlling costs and preventing overly long responses.

Let's see what happens when we ask for a summary with a very restrictive limit.

## `stop` - The Clean Ending

The `stop` parameter tells the model to stop generating as soon as it encounters a specific sequence of characters. This is perfect for generating lists or other structured data where you want to prevent extra conversational text.

Let's ask for a list of programming paradigms and stop after the first item.

## `n` - Generating Multiple Choices

The `n` parameter lets you get multiple different responses for the same prompt in a single API call. This is great for brainstorming when combined with a higher temperature.

## `response_format` - Guaranteed JSON

For developers, this is a game-changer. By setting `response_format={"type": "json_object"}`, you can force the model to output a syntactically correct JSON object. This is only available on newer models like `gpt-4o-mini` and is incredibly reliable.

Let's extract information from a sentence into a structured JSON object.